In [6]:
pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#To get results from LM Studio for only one file(Basic_Home_Remidied.pdf) using Langchain
import os
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Step 1: Set LM Studio local endpoint
os.environ["OPENAI_API_BASE"] = "http://127.0.0.1:4096/v1"
os.environ["OPENAI_API_KEY"] = "lm-studio"  # Any string works

# Load and process PDF
loader = PDFPlumberLoader("Basic_Home_Remedies.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))

# Chunking
# Note: This will download an embedding model on the first run
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

# Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})



# Step 2: Initialize the LLM (use the name of your loaded model)
llm = ChatOpenAI(
    model="llama-2-7b-langchain-chat",  # Matches your LM Studio
    temperature=0.7,
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    
    # --- THIS IS THE FIX ---
    # Give the model 5 minutes (300s) to respond instead of 60s
    request_timeout=300, 
    # ----------------------

    verbose=True
)

prompt = """
You are a domain expert assistant.
Use the provided context to answer the question clearly and accurately.
If the answer cannot be found in the context, say "The information is not available in the provided context."
Provide a well-structured answer in 3–4 sentences and keep it factual.

Context:
{context}

Question:
{question}

Answer:
"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query
result = qa("Give any Healthy Lifestyle Tips")
print("\n--- Final Answer ---")
print(result["result"])

Pages loaded: 3


C:\Users\Muskan\AppData\Local\Temp\ipykernel_4320\4093167303.py:107: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
C:\Users\Muskan\AppData\Local\Temp\ipykernel_4320\4093167303.py:111: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()




> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a domain expert assistant.
Use the provided context to answer the question clearly and accurately.
If the answer cannot be found in the context, say "The information is not available in the provided context."
Provide a well-structured answer in 3–4 sentences and keep it factual.

Context:
Context:
content:Healthy Lifestyle Tips
3.1 Balanced Diet
 Eat whole foods, including fruits, vegetables, whole grains, and lean proteins.  Reduce processed foods, sugar, and unhealthy fats.  Drink 8-10 glasses of water daily to stay hydrated.
source:Basic_Home_Remedies.pdf

Context:
content: Reduce intake of caffeinated and sugary drinks.  Avoid prolonged sitting; take breaks and move around. 4. Conclusion
Natural home remedies and a balanced lifestyle can help prevent and manage common ailments. A combination of healthy eating, regular exercise, stress management, and good sleep leads
to l